# Incremental vs Batch Training
This notebook copies each session directory one by one, runs incremental training, and logs evaluation metrics with wandb.

In [1]:
import os, shutil, time, datetime
import wandb
import torch
import gc
from incremental_training import main as incremental_main
from evaluate_holdout import evaluate_sampler_on_holdout

wandb.init(project='incremental_vs_batch')

Imports successful.
Visualization helpers defined.
Training and validation epoch functions adapted for Batch object and Denoiser.forward.


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: irvin-hwang (irvin-hwang-simulacra-systems) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
source_dir = r'C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_laundry_session_data_two_actions'
dest_dir = r'C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_laundry_session_data_two_actions_incremental_test'
os.makedirs(dest_dir, exist_ok=True)
sessions = sorted([d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))])

for session in sessions:
    src = os.path.join(source_dir, session)
    dst = os.path.join(dest_dir, session)
    if os.path.exists(dst):
        print(f'{dst} already exists, skipping copy')
        continue
    shutil.copytree(src, dst)
    print(f'Copied {src} -> {dst}')

    start = time.time()
    incremental_main()
    duration = str(datetime.timedelta(seconds=time.time() - start))
    print(f'Training took: {duration}')
    wandb.log({'session': session, 'training_duration': duration})

    gc.collect()
    torch.cuda.empty_cache()
    
    metrics, paths = evaluate_sampler_on_holdout()
    flat = {f'eval/{session}_{k}_{m}': v for k, vals in metrics.items() for m, v in vals.items()}
    wandb.log(flat)
    images = {f'eval/{session}_{name}': wandb.Image(path) for name, path in paths.items()}
    if images:
        wandb.log(images)

    gc.collect()
    torch.cuda.empty_cache()


wandb.finish()

C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_laundry_session_data_two_actions_incremental_test\session_20250528_094204 already exists, skipping copy
C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_laundry_session_data_two_actions_incremental_test\session_20250528_094602 already exists, skipping copy
Copied C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_laundry_session_data_two_actions\session_20250528_094947 -> C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_laundry_session_data_two_actions_incremental_test\session_20250528_094947
Found 2 new sessions to collect.


Wrote 9009 entries to C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_new_data_incremental_test\new.csv
Loaded combined CSV with columns: ['session_id', 'image_path', 'timestamp', 'action']
Fresh dataset size is 593.
Loaded combined CSV with columns: ['session_id', 'image_path', 'timestamp', 'action']
Full dataset size: 292
Loading dataset split from C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\output_model_2hz_DIAMOND_laundry_incremental_test\dataset_split.pth
Starting training
Incremental training for at least 12800 and at most 640000 with 640 number of steps per fresh data epoch


Incremental Training Steps:   0%|          | 0/640000 [00:00<?, ?it/s]

Filtering dataset with 30 samples for actions: [0.0]


Filtering Dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Filtered down to 9 samples.
Filtering dataset with 30 samples for actions: [0.13]


Filtering Dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Filtered down to 21 samples.
